# 2. Preparação e Tratamento de Dados


**Objetivo**: **Objetivo:** Limpar, tratar e enriquecer os dados selecionados na etapa anterior.

**Contexto (CRISP-DM):** Após o *Data Understanding*, onde identificamos anomalias (como dias de emprego), valores nulos e correlações, entramos na fase de *Data Preparation*. O output deste notebook será um dataset limpo (`df_cleaned.csv`) pronto para gerar insights de negócio.

 **Etapas deste Notebook:**
1.  **Tratamento de Anomalias:** Correção dos dias de emprego (365243) e datas negativas.
2.  **Análise de `OWN_CAR_AGE`:** Decisão sobre a remoção ou manutenção da coluna.
3.  **Imputação (Missing Values):** Tratamento lógico para `AMT_GOODS_PRICE`, `AMT_ANNUITY`, `EXT_SOURCE` e `OCCUPATION_TYPE`
4.  **Feature Engineering:** Criação das variáveis `AGE`, `DEBT_TO_INCOME`, `PAYMENT_RATE` e `DOWN_PAYMENT`.

## 2.1 Importação de Bibliotecas e Configurações

In [654]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Show all columns when displaying a DataFrame
pd.set_option('display.max_columns', None)  

## 2.2 Carga de Dados (Filtrados)

Carregamos apenas as colunas selecionadas segundo os 5 C's do Crédito.

In [655]:
# Separating only selected columns
selected_columns = [
    'TARGET', 'SK_ID_CURR',
    'NAME_CONTRACT_TYPE', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 
    'AMT_ANNUITY', 'AMT_GOODS_PRICE',
    'CODE_GENDER', 'CNT_CHILDREN', 'DAYS_BIRTH', 
    'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS',
    'DAYS_EMPLOYED', 'NAME_INCOME_TYPE', 'OCCUPATION_TYPE',
    'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'OWN_CAR_AGE',
    'REGION_RATING_CLIENT', 
    'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3'
]

df = pd.read_csv('../data/application_train.csv', usecols=selected_columns)
print(f"Dataset Loaded: {df.shape}")

Dataset Loaded: (307511, 22)


## 2.3 Tratamento de Anomalias e Formatação

Aqui teremos o tratamento para as seguintes variáveis:

### 2.2.1 Dias de Emprego (365243)
Conforme diagnóstico, o valor `365243` indica aposentados ou dados não preenchidos. Vamos substituir por `NaN` e transformar os dias negativos em positivos.

In [656]:
# 1. Replacing the annomaly data

print(f'Count of (NaN VALUES)/(365243 DATA) before replacement: ({df['DAYS_EMPLOYED'].isnull().sum()})/({df[df['DAYS_EMPLOYED'] == 365243].shape[0]})')

df['DAYS_EMPLOYED'] = df['DAYS_EMPLOYED'].replace(365243, np.nan)

print(f'Count of (NaN VALUES)/(365243 DATA) after replacement: ({df['DAYS_EMPLOYED'].isnull().sum()})/({df[df['DAYS_EMPLOYED'] == 365243].shape[0]})')
print('_' * 70)

#2. Changing time columns from negative to positive
time_cols=['DAYS_EMPLOYED', 'DAYS_BIRTH', 'OWN_CAR_AGE']

for col in time_cols:
    df[col] = df[col].abs()

print(f'Statistics for TIME COLUMNS after correction:')
df[time_cols].describe()

Count of (NaN VALUES)/(365243 DATA) before replacement: (0)/(55374)
Count of (NaN VALUES)/(365243 DATA) after replacement: (55374)/(0)
______________________________________________________________________
Statistics for TIME COLUMNS after correction:


,DAYS_EMPLOYED,DAYS_BIRTH,OWN_CAR_AGE
count,252137.000000,307511.000000,104582.000000
mean,2384.169325,16036.995067,12.061091
std,2338.360162,4363.988632,11.944812
min,0.000000,7489.000000,0.000000
25%,767.000000,12413.000000,5.000000
50%,1648.000000,15750.000000,9.000000
75%,3175.000000,19682.000000,15.000000
max,17912.000000,25229.000000,91.000000


## 2.4 Análise de Remoção: OWN_CAR_AGE

O roteiro previa analisar a remoção desta coluna devido à alta taxa de nulos.
Vamos verificar a porcentagem de nulos e a relação com a posse de carro.

Aqui, como temos outliers muito elevados para `AMT_ANNUITY` e `AMT_GOODS_PRICE` teremos que aplicar a escala logaritmica para melhorar a visualização da distribuição.

In [657]:
# Verifiying null percentage
null_val = df['OWN_CAR_AGE'].isnull().sum()
print(f'Number of null values in OWN_CAR_AGE: {null_val}')

#2. Checking consistencies:
#Condition: No Car (N) and Age is Null
mask_consistent = len(df[(df['FLAG_OWN_CAR'] == 'N') & (df['OWN_CAR_AGE'].isna())])
print(f'Of these null values in OWN_CAR_AGE, how many are from people who do not own a car? {mask_consistent} ({(mask_consistent / null_val) * 100:.3f}%)')

Number of null values in OWN_CAR_AGE: 202929
Of these null values in OWN_CAR_AGE, how many are from people who do not own a car? 202924 (99.998%)


**Veredito:**
Observamos que os valores nulos em `OWN_CAR_AGE` não são "erros", mas sim uma **característica estrutural**: quem não tem carro, não tem idade do carro.

**Decisão:** **NÃO removeremos a coluna.**
Motivo: Remover a coluna eliminaria a informação de "idade do veículo" para quem *tem* carro, que pode ser um indicador de patrimônio. Manteremos os nulos como estão (para modelos de árvore) ou imputaremos 0 em modelos lineares futuramente.

Ainda assim, encontramos 5 casos (202929 - 202924) inconsistentes de clientes que afirmam ter carro (`FLAG_OWN_CAR` = 'Y), mas a idade do veículo (`OWN_CAR_AGE`) é nula. 
Para não descartar esses clientes (que podem conter dados valiosos de risco), preencheremos esses 5 casos com a **mediana** da idade dos carros da base.

### 2.3.1 Correção de Inconsistência: Carro sem Idade

In [658]:
# 1. Median of existing cars
car_age_median = df['OWN_CAR_AGE'].median()

#2. Find the inconsistencies and fill them in
#Condition: Has a car (Y) and age is null
mask_inconsistent = (df['FLAG_OWN_CAR'] == 'Y') & (df['OWN_CAR_AGE'].isnull())
print(f"Inconsistencies found: {mask_inconsistent.sum()}")

# 3. Replace selcted lines
df.loc[mask_inconsistent, 'OWN_CAR_AGE'] = car_age_median

print(f"Remaining inconsistencies: {((df['FLAG_OWN_CAR'] == 'Y') & (df['OWN_CAR_AGE'].isnull())).sum()}")

Inconsistencies found: 5
Remaining inconsistencies: 0


## 2.4 Imputação (Preenchimento de Nulos)

 * **AMT_GOODS_PRICE:** Usaremos `AMT_CREDIT` (correlação 0.99).
 * **AMT_ANNUITY:** Usaremos a **Mediana** (Para evitar distorção de escala, não podemos usar o valor do crédito direto, pois o crédito é ~20x maior que a parcela).
 * **EXT_SOURCE:**: Mantemos NaN conforme planejado
 * **OCCUPATION_TYPE**: Analisaremos inconsistências para preencher corretamente

In [659]:
# %%
# Imputing GOODS_PRICE with CREDIT
df['AMT_GOODS_PRICE'].fillna(df['AMT_CREDIT'], inplace=True)

# Imputing ANNUITY with MEDIAN
df['AMT_ANNUITY'].fillna(df['AMT_ANNUITY'].median(), inplace=True)


print("Allocation performed. Remaining null values in GOODS_PRICE:", df['AMT_GOODS_PRICE'].isnull().sum())

Allocation performed. Remaining null values in GOODS_PRICE: 0


C:\Users\lucas\AppData\Local\Temp\ipykernel_29796\1380837074.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['AMT_GOODS_PRICE'].fillna(df['AMT_CREDIT'], inplace=True)
C:\Users\lucas\AppData\Local\Temp\ipykernel_29796\1380837074.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a co

### 2.4.1 Análise de OCCUPATION_TYPE:

Detectamos um alto volume de nulos nesta coluna. Iremos preencher seguindo a seguinte lógica de negócio:

1.  **Grupo 'Unknown' (Risco de Omissão):** Se a pessoa afirma que trabalha (`NAME_INCOME_TYPE` = Working, State servant, etc.), mas não informou a profissão, classificaremos como `Unknown`. Isso pode indicar informalidade.
2.  **Grupo 'Not Applicable' (Estrutural):** Se a pessoa é Aposentada ou Desempregada, a ausência de profissão é natural. Classificaremos como `NA`.

In [660]:
# Analyzing the possible values for the two columns
print(df['NAME_INCOME_TYPE'].value_counts())
print('_'*20)
print(df['OCCUPATION_TYPE'].value_counts())

NAME_INCOME_TYPE
Working                 158774
Commercial associate     71617
Pensioner                55362
State servant            21703
Unemployed                  22
Student                     18
Businessman                 10
Maternity leave              5
Name: count, dtype: int64
____________________
OCCUPATION_TYPE
Laborers                 55186
Sales staff              32102
Core staff               27570
Managers                 21371
Drivers                  18603
High skill tech staff    11380
Accountants               9813
Medicine staff            8537
Security staff            6721
Cooking staff             5946
Cleaning staff            4653
Private service staff     2652
Low-skill Laborers        2093
Waiters/barmen staff      1348
Secretaries               1305
Realty agents              751
HR staff                   563
IT staff                   526
Name: count, dtype: int64


In [661]:
# Defining which types of income imply having a profession
working_income_types = [
    'Working',
    'State servant',
    'Commercial associate',
    'Businessman',
    'Maternity leave' # Generally maintains employment relationship
]

# Function to apply logic
def impute_occupation(row):
    # If already have a job, keep it
    if pd.notna(row['OCCUPATION_TYPE']): # All lines that has some data (!= NaN)
        return row['OCCUPATION_TYPE']

    # If is null, check the income source
    if row['NAME_INCOME_TYPE'] in working_income_types:
        return 'Unknown' # He Works but didn't say what he does
    else:
        return 'Not Applicable' # Unenployed, Pensioner, Student 

# Applying transformation
df['OCCUPATION_TYPE'] = df.apply(impute_occupation, axis=1)

# Checking new distribution
print(f'Null values after APPLY: {df['OCCUPATION_TYPE'].isnull().sum()}')
print("Nova distribuição de OCCUPATION_TYPE:")
print(df['OCCUPATION_TYPE'].value_counts())

Null values after APPLY: 0
Nova distribuição de OCCUPATION_TYPE:
OCCUPATION_TYPE
Not Applicable           55384
Laborers                 55186
Unknown                  41007
Sales staff              32102
Core staff               27570
Managers                 21371
Drivers                  18603
High skill tech staff    11380
Accountants               9813
Medicine staff            8537
Security staff            6721
Cooking staff             5946
Cleaning staff            4653
Private service staff     2652
Low-skill Laborers        2093
Waiters/barmen staff      1348
Secretaries               1305
Realty agents              751
HR staff                   563
IT staff                   526
Name: count, dtype: int64


## 2.5 Crianção de novas variáveis (Feature Engineering)

Criaremos as variáveis derivadas planejadas para enriquecer a análise de risco.

 1.  `AGE`: Idade em Anos.
 2.  `DEBT_TO_INCOME_RATIO`: Comprometimento da renda (`ANNUITY / INCOME`).
 3.  `PAYMENT_RATE`: Peso da parcela sobre a dívida total (`ANNUITY / CREDIT`).
 4.  `DOWN_PAYMENT`: Entrada estimada (`GOODS - CREDIT`).

---
* `AGE`

In [662]:
# 1. Age (Years)
df['AGE'] = df['DAYS_BIRTH'] / 365
print(f'Null values for AGE = {df['AGE'].isnull().sum()}')

Null values for AGE = 0


---
* `DEBT_TO_INCOME_HOME`

In [663]:
# 2. DEBT_TO_INCOME_RATIO (Income Commitment)
# The higher it is, the greater the theoretical risk (customer default).
df['DEBT_TO_INCOME_RATIO'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
print(f'Null values for DEBT_TO_INCOME_RATIO = {df['DEBT_TO_INCOME_RATIO'].isnull().sum()}')

Null values for DEBT_TO_INCOME_RATIO = 0


---
- `CREDIT_TERM`: 

In [664]:
# 3. PAYMENT_RATE
# Indicates the speed of payment. Very low rates indicate very long terms.
df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
print(f'Null values for PAYMENT_RATE = {df['PAYMENT_RATE'].isnull().sum()}')

Null values for PAYMENT_RATE = 0


---
- `DOWN_PAYMENT`

In [665]:
# 4. DOWN_PAYMENT
# If GOODS > CREDIT, the customer paid the difference (Positive Down Payment).
# If GOODS < CREDIT, the customer took extra cash (Negative Down Payment/Cash Out).
df['DOWN_PAYMENT'] = df['AMT_GOODS_PRICE'] - df['AMT_CREDIT']

In [666]:
# Final feature
new_features = ['AGE', 'DEBT_TO_INCOME_RATIO', 'PAYMENT_RATE', 'DOWN_PAYMENT']
df[new_features].describe()

,AGE,DEBT_TO_INCOME_RATIO,PAYMENT_RATE,DOWN_PAYMENT
count,307511.000000,307511.000000,307511.000000,307511.000000
mean,43.936973,0.180929,0.053695,-60863.724220
std,11.956133,0.094573,0.022482,70487.026661
min,20.517808,0.000224,0.016790,-540000.000000
25%,34.008219,0.114782,0.036900,-99792.000000
50%,43.150685,0.162833,0.050000,-39204.000000
75%,53.923288,0.229067,0.064043,0.000000
max,69.120548,1.875965,0.158114,765000.000000


## 2.6 Conclusão e Exportação

O dataset foi tratado conforme o planejamento:
* Anomalias de tempo removidas.
* `OWN_CAR_AGE` mantida conscientemente.
* Nulos críticos preenchidos.
* Novas features de negócio criadas.

### Próxima Etapa: Validação de Hipóteses (Notebook 03)
Agora usaremos essas novas variáveis (`AGE`, `DEBT_TO_INCOME`, etc.) para responder perguntas de negócio e encontrar perfis de risco.

In [667]:
# Ensuring the folder exists
os.makedirs('../data/processed', exist_ok=True) 

# Saving data
df.to_csv('../data/processed/application_train_cleaned.csv', index=False)
print("Arquivo 'application_train_cleaned.csv' exportado com sucesso.")

Arquivo 'application_train_cleaned.csv' exportado com sucesso.
